In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv', index_col=0)
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv', index_col=0)
submission_df = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv', index_col=0)
train_df

In [ ]:
train_df.info()

In [ ]:
train_df.describe().T

In [ ]:
train_df.describe(include='object').T

In [ ]:
X = train_df.drop(['Name', 'Ticket', 'Cabin', 'Survived'], axis=1)
y = train_df['Survived']
test_df = test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
X.describe(include='all')

In [ ]:
num_cols = [col for col in X.columns if X[col].dtype == 'float64']
cat_cols = [col for col in X.columns if X[col].dtype == 'int64' or X[col].dtype == 'object']
print('categorical columns:', cat_cols)
print('numerical columns:', num_cols)

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


num_trasform = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='mean')),
    ('num_scale', MinMaxScaler())
])

cat_transrom = Pipeline(steps=[
    ('cat_impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [ ]:
from sklearn.compose import ColumnTransformer

full_preprocessing = ColumnTransformer(transformers=[
    ('num', num_trasform, num_cols),
    ('cat', cat_transrom, cat_cols)
])

In [ ]:
from xgboost import XGBClassifier

xgb_pip = Pipeline(steps=[
    ('preprocess', full_preprocessing),
    ('model', XGBClassifier(n_estimator=20, learning_rate=0.05, max_depth=6, use_label_encoder=False, n_jobs=-1))
])

In [ ]:
# from sklearn.model_selection import GridSearchCV

# params = {'model__learning_rate': np.arange(0.05, 0.6, 0.2), 
#           'model__n_estimator': [20, 50, 100],
#           'model__max_depth': [3, 6, 9]
#          }

# search = GridSearchCV(xgb_pip, params,
#                      cv=5,
#                       scoring='accuracy'
#                      )
# _ = search.fit(X, y)

In [ ]:
# search.best_params_
# output: {'model__learning_rate': 0.05, 'model__max_depth': 6, 'model__n_estimator': 20}

In [ ]:
xgb_pip.fit(X, y)
final_pred = xgb_pip.predict(test_df)

In [ ]:
submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': final_pred})
submission.to_csv('submission.csv', index=False)
print('All done well!')